In [36]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
%matplotlib inline
from classifiers import *

In [37]:
train_data = pd.read_csv('data/train.csv', sep = '|')
test_data = pd.read_csv('data/test.csv', sep = '|')
print(f'Train set has {train_data.shape[0]} entries and {train_data.shape[1]} features')
print(f'Test set has {test_data.shape[0]} entries and {test_data.shape[1]} features')
# print(f'attributes: \n {test_data.columns}')
# print(f'types: \n {train_data.dtypes}')

Train set has 1879 entries and 10 features
Test set has 498121 entries and 9 features


In [38]:
# encode category feature
X_train = pd.get_dummies(train_data, columns=['trustLevel'], prefix='trustLevel')
X_train = X_train.drop(columns=['fraud'])
X_test = pd.get_dummies(test_data, columns=['trustLevel'], prefix='trustLevel')
y = train_data['fraud']

In [39]:
# normalize 
X = X_train.append(X_test)
X_train_norm = StandardScaler().fit_transform(X)
X_norm = X_train_norm[:1879,:]
X_test_norm = X_train_norm[1879:,:]


/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/usr/local/lib/python3.7/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [ ]:
evaluate_classification(X_norm, y)
# positive class is not predicted in Random Forest => should check balance labels

Naive Bayes: test core = -11340 
K Nerest Neighbors: test core = -1700 
Linear SVM: test core = -520 
RBF SVM: test core = -520 


/usr/local/lib/python3.7/site-packages/sklearn/gaussian_process/gpc.py:413: RuntimeWarning: overflow encountered in exp
  - np.log(np.diag(L)).sum()


<h2> Try xgboost with cost sensitive </h2>

In [32]:
#!/usr/bin/python
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def logistic_obj(y_hat, dtrain, alpha=5, beta=25): # alpha for FN beta for FP
    y = dtrain.get_label()
    pred = 1. / (1. + np.exp(-y_hat))
    grad = pred * (beta + alpha*y - beta*y) - alpha*y  # alpha*(p-1)*y + beta*p*(1-y)
    hess = pred * (1 - pred) * (beta + alpha*y - beta*y)
    return grad, hess

def err_rate(pred, dtrain):
    y = dtrain.get_label()
    pred = 1. / (1. + np.exp(-pred))
    loss_fn = y*np.log(pred)
    loss_fp = (1.0 - y)*np.log(1.0 - pred)
    return 'error', np.sum(-(5*loss_fn+25*loss_fp))/len(y)



In [48]:
# prepare input data
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=42)

def cross_validate(X_train, X_test, y_train, y_test,depth):
    # load data
    d_train = xgb.DMatrix(X_train, label=y_train)
    d_val = xgb.DMatrix(X_test, label=y_test)
    # build model
    param = {'max_depth': depth, 'eta': 1, 'silent': 1, 'seed': 0}
    watchlist = [(d_val, 'eval'), (d_train, 'train')]
    num_round = 10
    model_trn = xgb.train(param, d_train, num_round, watchlist, obj=logistic_obj, feval=err_rate)
#     model_trn = xgb.cv(param, d_train, num_round, nfold=10, obj=logistic_obj, feval=profit_scorer)

    print(model_trn)
    # prediction
    pred = model_trn.predict(d_val) 
    pred = 1. / (1. + np.exp(-pred))
#     print(pred)
    return pred

pred = cross_validate(X_train, X_test, y_train, y_test,10)
print(f'cost = {profit_scorer(y_test, pred>0.5)}')

[0]	eval-rmse:2.13458	train-rmse:2.10701	eval-error:3.60496	train-error:2.82673
[1]	eval-rmse:3.19775	train-rmse:3.17288	eval-error:2.72454	train-error:0.984047
[2]	eval-rmse:4.16468	train-rmse:4.17326	eval-error:2.06821	train-error:0.359491
[3]	eval-rmse:5.09802	train-rmse:5.12145	eval-error:1.54672	train-error:0.149323
[4]	eval-rmse:6.01971	train-rmse:6.04156	eval-error:1.54653	train-error:0.064146
[5]	eval-rmse:6.87529	train-rmse:6.90585	eval-error:1.43376	train-error:0.03263
[6]	eval-rmse:7.68033	train-rmse:7.70648	eval-error:1.37229	train-error:0.019419
[7]	eval-rmse:8.30954	train-rmse:8.34334	eval-error:1.33709	train-error:0.013175
[8]	eval-rmse:8.84312	train-rmse:8.86081	eval-error:1.21554	train-error:0.010303
[9]	eval-rmse:9.11996	train-rmse:9.16637	eval-error:1.2562	train-error:0.008685
cost = -130


In [49]:
for i in range(10):
    pred = cross_validate(X_train, X_test, y_train, y_test,i)
    print(f'depth {i+1} = {profit_scorer(y_test, pred>0.5)}')

[0]	eval-rmse:2.14601	train-rmse:2.168	eval-error:3.25119	train-error:3.40013
[1]	eval-rmse:3.13491	train-rmse:3.15575	eval-error:1.73549	train-error:2.00125
[2]	eval-rmse:3.87272	train-rmse:3.89306	eval-error:1.33509	train-error:1.67685
[3]	eval-rmse:4.28143	train-rmse:4.3016	eval-error:1.24835	train-error:1.63015
[4]	eval-rmse:4.38731	train-rmse:4.40742	eval-error:1.23618	train-error:1.6282
[5]	eval-rmse:4.39341	train-rmse:4.4135	eval-error:1.23559	train-error:1.62819
[6]	eval-rmse:4.39342	train-rmse:4.41358	eval-error:1.23558	train-error:1.62819
[7]	eval-rmse:4.39342	train-rmse:4.41358	eval-error:1.23558	train-error:1.62819
[8]	eval-rmse:4.39342	train-rmse:4.41358	eval-error:1.23558	train-error:1.62819
[9]	eval-rmse:4.39342	train-rmse:4.41358	eval-error:1.23558	train-error:1.62819
depth 1 = -600
[0]	eval-rmse:2.13907	train-rmse:2.15315	eval-error:3.2253	train-error:3.3436
[1]	eval-rmse:3.15016	train-rmse:3.15193	eval-error:1.66132	train-error:1.84188
[2]	eval-rmse:4.03066	train-rmse

[5]	eval-rmse:6.89047	train-rmse:6.91206	eval-error:1.16335	train-error:0.031579
[6]	eval-rmse:7.68936	train-rmse:7.70249	eval-error:1.10817	train-error:0.019347
[7]	eval-rmse:8.3477	train-rmse:8.36133	eval-error:1.10576	train-error:0.013014
[8]	eval-rmse:8.74122	train-rmse:8.77916	eval-error:1.06129	train-error:0.010108
[9]	eval-rmse:9.29809	train-rmse:9.32223	eval-error:1.02896	train-error:0.00834
depth 10 = -145
